# Day 15

In [460]:
data = [7,14,0,17,11,1,2].freeze

[7, 14, 0, 17, 11, 1, 2]

In [461]:
ocache = data.reverse.drop(1).reverse.each_with_index.to_h.freeze

{7=>0, 14=>1, 0=>2, 17=>3, 11=>4, 1=>5}

In [463]:
cache=Marshal.load(Marshal.dump(ocache))
n=data.last

((data.count - 1)...(2020 - 1)).each do |i|
  if v=cache[n]
    next_val = i - v
    cache[n] = i
    n = next_val
  else
    cache[n] = i
    n = 0
  end

  if i % 1_000_000 == 0
    print "/"
  end
end
puts n

206


## Easy performance improvements

I used the above to get the part 2 solution, and a version where the cache contained two rolling values for each number. It took around 15-30 seconds per million. :/


Much of the time spent during the loop is spent accessing the lookup hash in the global scope
Moving the variables more local improves cost significantly, event with the added price of
additional array creation etc.

In [479]:
require 'benchmark'

cache=Marshal.load(Marshal.dump(ocache))
n=data.last

def in_method(count, cache, n)
  count.times do |i|
    cache[n]
  end
end

Benchmark.bm(15) do |x|
  x.report("Global lookup") do
    1_000_000.times do
      cache[n]
    end
  end

  x.report("Local lookup") do
    1_000_000.times.inject(cache) do |cache|
      cache[n]
      
      cache
    end
  end
  
  x.report("Local key") do
    1_000_000.times.inject(n) do |n|
      cache[n]
      
      n
    end
  end
  
  x.report("lookup + key") do
    1_000_000.times.inject([cache, n]) do |c, n|
      c[n]
      
      [c, n]
    end
  end
  
  x.report("Method scope") do
    in_method(1_000_000, cache, n)
  end 
end;nil

                      user     system      total        real
Global lookup     8.420360   0.000000   8.420360 (  8.429577)
Local lookup      4.282466   0.003861   4.286327 (  4.290906)
Local key         4.299350   0.000000   4.299350 (  4.303004)
lookup + key      0.099704   0.000000   0.099704 (  0.099510)
Method scope      0.061123   0.000000   0.061123 (  0.061131)


In [482]:
cache=Marshal.load(Marshal.dump(ocache))
n=data.last

def run(cache, n, starting, ending)
  (starting...ending).each do |i|
    if v=cache[n]
      next_val = i - v
      cache[n] = i
      n = next_val
    else
      cache[n] = i
      n = 0
    end
  end
  n
end

puts Benchmark.measure {
  n=run(cache, n, data.count-1, 30_000_000-1)
}
n

  6.201681   0.023926   6.225607 (  6.236146)



955

### Enumerator

Slower, but nicer interface

In [551]:
def memory_game(data)
  Enumerator.new do |yielder|
    all_but_last = data.reverse.drop(1).reverse
    cache = all_but_last.each_with_index.to_h

    data.each {|i| yielder.yield(i) }
    n = data.last

    loop.with_index(data.count - 1) do |_, i|
      if v=cache[n]
        next_val = i - v
        cache[n] = i
        n = next_val
      else
        cache[n] = i
        n = 0
      end
      
      yielder.yield(n)
    end
  end
end

Benchmark.bm do |x|
  index = 30_000_000 - 1
  
  x.report("next") do
    sequence = memory_game(data)
    index.times { sequence.next }
    n = sequence.next
  end
  
  x.report("find") do
    n = memory_game(data).find.with_index {|_n, i| i == index }
  end
  
  x.report("drop") do
    n = memory_game(data).lazy.drop(index).first
  end
end;nil

       user     system      total        real
next 19.691053   0.143800  19.834853 ( 19.859294)
find 13.011795   0.011868  13.023663 ( 13.045248)
drop 13.300430   0.007862  13.308292 ( 13.330321)
